In [80]:
from rdflib import Graph
from gensim.parsing.preprocessing import STOPWORDS
from om.ont import split_entity
from om.ont import get_n
from rdflib import Namespace, URIRef, Literal, BNode
from tqdm import tqdm
import os

In [56]:
base = '/home/guilherme/IdeaProjects/conference-dataset-population-elodie/populated_datasets/data_100'


In [64]:
!ls /home/guilherme/IdeaProjects/conference-dataset-population-elodie/populated_datasets/data_100

cmt_100.ttl  conference_100.ttl  confOf_100.ttl  edas_100.ttl  ekaw_100.ttl


In [83]:
def filtern(x, g1):
    if type(x) == URIRef:
        n = get_n(x, g1)
        words = split_entity(n)
        words = map(lambda x: x.lower(), words)
        return set(filter(lambda x: x not in STOPWORDS and not x.isdigit(), words))

    elif type(x) == Literal:
        words = split_entity(x)
        words = map(lambda x: x.lower(), words)
        return set(filter(lambda x: x not in STOPWORDS and not x.isdigit(), words))

    return set()

In [84]:
for p, d, f in os.walk(base):

    for file in f:
        on = file.split('_')[0]

        g1 = Graph()
        g1.parse(base + '/' + file)
        g1.bind(f'{on}-instances', Namespace(f'http://{on}-instances#'))


        sents = dict()

        for s, p, o in tqdm(g1):

            sents.setdefault(s, set()).update(filtern(s, g1))
            sents.setdefault(s, set()).update(filtern(p, g1))
            sents.setdefault(s, set()).update(filtern(o, g1))


        lines = []

        for k in sents:
            line = ' '.join(sents[k]) + ' #' + str(k) + '\n'
            lines.append(line)

        with open('starsp.txt', 'a') as f:
            f.writelines(lines)

100%|██████████| 248595/248595 [00:12<00:00, 19167.92it/s]
